# Análise Descritiva Completa do Setor Vitivinícola Brasileiro

## Objetivo
Análise descritiva integrada dos dados de exportação de vinhos brasileiros utilizando todas as bases de dados disponíveis:
- Quantidade de vinhos exportados por país de destino
- Fatores externos que interferem nas análises
- Correlações entre produção, comercialização, processamento e exportação
- Montante de vendas de exportação nos últimos 15 anos
- Análise por país e identificação de oportunidades

In [18]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## 1. Carregamento e Preparação de Todas as Bases de Dados

In [19]:
# Carregamento de todos os arquivos
df_producao = pd.read_csv('Database/Producao.csv', sep=';')
df_comercializacao = pd.read_csv('Database/Comercializacao.csv', sep=';')
df_processamento = pd.read_csv('Database/Processamento.csv', sep=';')
df_exportacao = pd.read_csv('Database/Exportacao.csv', sep=';')
df_importacao = pd.read_csv('Database/Importacao.csv', sep=';')

print("ESTRUTURA DAS BASES DE DADOS:")
print(f"Produção: {df_producao.shape[0]} registros, {df_producao.shape[1]} colunas")
print(f"Comercialização: {df_comercializacao.shape[0]} registros, {df_comercializacao.shape[1]} colunas")
print(f"Processamento: {df_processamento.shape[0]} registros, {df_processamento.shape[1]} colunas")
print(f"Exportação: {df_exportacao.shape[0]} registros, {df_exportacao.shape[1]} colunas")
print(f"Importação: {df_importacao.shape[0]} registros, {df_importacao.shape[1]} colunas")

# Período de análise (últimos 15 anos)
anos_analise = [str(ano) for ano in range(2009, 2024)]
print(f"\nPeríodo de análise: 2009-2023 ({len(anos_analise)} anos)")

ESTRUTURA DAS BASES DE DADOS:
Produção: 51 registros, 57 colunas
Comercialização: 62 registros, 57 colunas
Processamento: 139 registros, 57 colunas
Exportação: 137 registros, 110 colunas
Importação: 68 registros, 110 colunas

Período de análise: 2009-2023 (15 anos)


## 2. Consolidação dos Dados Anuais

In [20]:
# Função para consolidar dados anuais
def consolidar_dados_anuais(df, anos, excluir_brasil=False):
    if excluir_brasil and 'País' in df.columns:
        df = df[df['País'] != 'Brasil']
    
    dados_consolidados = {}
    for ano in anos:
        if ano in df.columns:
            total = pd.to_numeric(df[ano], errors='coerce').fillna(0).sum()
            dados_consolidados[int(ano)] = total
        else:
            dados_consolidados[int(ano)] = 0
    
    return dados_consolidados

# Consolidar dados de todas as bases
dados_producao = consolidar_dados_anuais(df_producao, anos_analise)
dados_comercializacao = consolidar_dados_anuais(df_comercializacao, anos_analise)
dados_processamento = consolidar_dados_anuais(df_processamento, anos_analise)
dados_exportacao = consolidar_dados_anuais(df_exportacao, anos_analise, excluir_brasil=True)
dados_importacao = consolidar_dados_anuais(df_importacao, anos_analise)

# Criar DataFrame consolidado
df_consolidado = pd.DataFrame({
    'Ano': list(range(2009, 2024)),
    'Producao_Litros': [dados_producao[ano] for ano in range(2009, 2024)],
    'Comercializacao_Litros': [dados_comercializacao[ano] for ano in range(2009, 2024)],
    'Processamento_Kg': [dados_processamento[ano] for ano in range(2009, 2024)],
    'Exportacao_Litros': [dados_exportacao[ano] for ano in range(2009, 2024)],
    'Importacao_Litros': [dados_importacao[ano] for ano in range(2009, 2024)]
})

print("DADOS CONSOLIDADOS (2009-2023):")
print(df_consolidado)

# Estatísticas básicas
print("\nESTATÍSTICAS BÁSICAS DO PERÍODO:")
for coluna in ['Producao_Litros', 'Comercializacao_Litros', 'Exportacao_Litros', 'Importacao_Litros']:
    total = df_consolidado[coluna].sum()
    media = df_consolidado[coluna].mean()
    print(f"{coluna.replace('_', ' ')}: Total = {total:,.0f}, Média anual = {media:,.0f}")

DADOS CONSOLIDADOS (2009-2023):
     Ano  Producao_Litros  Comercializacao_Litros  Processamento_Kg  \
0   2009        683643528               679550353       143410052.0   
1   2010        642820786               643306862        91475906.0   
2   2011        921104534               672553986       164809206.0   
3   2012        857603200               643569389       151646074.0   
4   2013        743277338               720817130       147627520.0   
5   2014        748536510               687248805       131159722.0   
6   2015        885403428               712369284       140132408.0   
7   2016        401100688               583810281        64280464.0   
8   2017        971663578               610735252       153818256.0   
9   2018        835505726               648636606       131181340.0   
10  2019        805878516               833736865               0.0   
11  2020        637024382               892823595       138015442.0   
12  2021        974677500               91593

## 3. Tabela Principal: Exportações por País de Destino

In [21]:
# Preparar dados de exportação por país (2009-2023)
df_exp_paises = df_exportacao[df_exportacao['País'] != 'Brasil'].copy()

# Calcular total por país no período
df_exp_paises['Total_Periodo'] = 0
for ano in anos_analise:
    if ano in df_exp_paises.columns:
        df_exp_paises['Total_Periodo'] += pd.to_numeric(df_exp_paises[ano], errors='coerce').fillna(0)

# Filtrar países com exportações significativas
df_paises_ativos = df_exp_paises[df_exp_paises['Total_Periodo'] > 0].copy()
df_paises_ativos = df_paises_ativos.sort_values('Total_Periodo', ascending=False)

# Criar tabela conforme solicitado
preco_medio_litro = 4.20  # US$ por litro (estimativa baseada em dados de mercado)

tabela_exportacao = pd.DataFrame({
    'Pais_Origem': 'Brasil',
    'Pais_Destino': df_paises_ativos['País'],
    'Quantidade_Litros': df_paises_ativos['Total_Periodo'].astype(int),
    'Valor_USD_Estimado': (df_paises_ativos['Total_Periodo'] * preco_medio_litro).round(2)
})

print("TABELA DE EXPORTAÇÕES DE VINHO BRASILEIRO (2009-2023)")
print("País de Origem: Brasil")
print("Conversão: 1 KG = 1 Litro")
print(f"Preço médio estimado: US$ {preco_medio_litro} por litro")
print("=" * 80)
print(f"{'Rank':<4} {'País Destino':<25} {'Quantidade (L)':<15} {'Valor (US$)':<15} {'%':<6}")
print("-" * 80)

total_exportacao = tabela_exportacao['Quantidade_Litros'].sum()
for i, (_, row) in enumerate(tabela_exportacao.head(20).iterrows(), 1):
    pais = row['Pais_Destino']
    quantidade = row['Quantidade_Litros']
    valor = row['Valor_USD_Estimado']
    percentual = (quantidade / total_exportacao) * 100
    print(f"{i:<4} {pais:<25} {quantidade:>12,.0f} {valor:>12,.0f} {percentual:>5.1f}%")

print(f"\nTOTAL GERAL: {total_exportacao:,.0f} litros | US$ {total_exportacao * preco_medio_litro:,.0f}")
print(f"Número de países importadores: {len(df_paises_ativos)}")

TABELA DE EXPORTAÇÕES DE VINHO BRASILEIRO (2009-2023)
País de Origem: Brasil
Conversão: 1 KG = 1 Litro
Preço médio estimado: US$ 4.2 por litro
Rank País Destino              Quantidade (L)  Valor (US$)     %     
--------------------------------------------------------------------------------
1    Rússia                      32,822,141  137,852,992  39.5%
2    Paraguai                    30,803,247  129,373,637  37.0%
3    Estados Unidos               3,349,299   14,067,056   4.0%
4    China                        2,574,686   10,813,681   3.1%
5    Haiti                        2,351,228    9,875,158   2.8%
6    Espanha                      1,990,238    8,359,000   2.4%
7    Reino Unido                  1,150,780    4,833,276   1.4%
8    Uruguai                      1,118,688    4,698,490   1.3%
9    Japão                          972,341    4,083,832   1.2%
10   Países Baixos                  897,986    3,771,541   1.1%
11   Alemanha, República Democrática      648,115    2,722,083   0

## 4. Análise da Cadeia Produtiva Integrada

In [22]:
# Calcular indicadores da cadeia produtiva
df_consolidado['Balanca_Comercial'] = df_consolidado['Exportacao_Litros'] - df_consolidado['Importacao_Litros']
df_consolidado['Taxa_Exportacao'] = (df_consolidado['Exportacao_Litros'] / df_consolidado['Producao_Litros']) * 100
df_consolidado['Eficiencia_Comercial'] = (df_consolidado['Comercializacao_Litros'] / df_consolidado['Producao_Litros']) * 100
df_consolidado['Consumo_Aparente'] = (df_consolidado['Producao_Litros'] + 
                                     df_consolidado['Importacao_Litros'] - 
                                     df_consolidado['Exportacao_Litros'])

print("INDICADORES DA CADEIA PRODUTIVA (2009-2023):")
print("=" * 70)
print(f"{'Ano':<6} {'Produção':<12} {'Export.':<10} {'Import.':<10} {'Taxa Exp%':<10} {'Bal.Com.':<12}")
print("-" * 70)

for _, row in df_consolidado.iterrows():
    ano = int(row['Ano'])
    producao = row['Producao_Litros']
    exportacao = row['Exportacao_Litros']
    importacao = row['Importacao_Litros']
    taxa_exp = row['Taxa_Exportacao']
    balanca = row['Balanca_Comercial']
    
    print(f"{ano:<6} {producao:>10,.0f} {exportacao:>8,.0f} {importacao:>8,.0f} "
          f"{taxa_exp:>8.2f}% {balanca:>10,.0f}")

# Análise de correlações
correlacoes = df_consolidado[['Producao_Litros', 'Comercializacao_Litros', 
                             'Exportacao_Litros', 'Importacao_Litros']].corr()

print("\nCORRELAÇÕES ENTRE VARIÁVEIS DA CADEIA:")
print("=" * 45)
print(correlacoes.round(3))

# Principais achados
taxa_exp_media = df_consolidado['Taxa_Exportacao'].mean()
anos_superavit = len(df_consolidado[df_consolidado['Balanca_Comercial'] > 0])
anos_deficit = len(df_consolidado[df_consolidado['Balanca_Comercial'] < 0])

print(f"\nPRINCIPAIS ACHADOS:")
print(f"Taxa média de exportação: {taxa_exp_media:.2f}% da produção")
print(f"Anos com superávit comercial: {anos_superavit}")
print(f"Anos com déficit comercial: {anos_deficit}")
print(f"Correlação Produção x Exportação: {correlacoes.loc['Producao_Litros', 'Exportacao_Litros']:.3f}")

INDICADORES DA CADEIA PRODUTIVA (2009-2023):
Ano    Produção     Export.    Import.    Taxa Exp%  Bal.Com.    
----------------------------------------------------------------------
2009   683,643,528 25,514,198 55,926,957     3.73% -30,412,759
2010   642,820,786 1,280,574 70,737,868     0.20% -69,457,294
2011   921,104,534 1,214,834 72,705,225     0.13% -71,490,391
2012   857,603,200 5,775,376 74,209,440     0.67% -68,434,064
2013   743,277,338 9,149,059 67,954,436     1.23% -58,805,377
2014   748,536,510 2,324,029 76,910,155     0.31% -74,586,126
2015   885,403,428 1,198,479 77,686,474     0.14% -76,487,995
2016   401,100,688 1,786,549 88,345,068     0.45% -86,558,519
2017   971,663,578 2,889,572 118,330,745     0.30% -115,441,173
2018   835,505,726 3,856,162 109,971,001     0.46% -106,114,839
2019   805,878,516 3,152,873 114,175,423     0.39% -111,022,550
2020   637,024,382 4,409,717 147,135,137     0.69% -142,725,420
2021   974,677,500 8,058,673 154,690,551     0.83% -146,631,878
2

## 5. Análise por Categorias de Produtos

In [23]:
# Análise da produção por categoria (2023)
categorias_producao = {
    'VINHO DE MESA': df_producao[df_producao['control'] == 'VINHO DE MESA'],
    'VINHO FINO DE MESA (VINIFERA)': df_producao[df_producao['control'] == 'VINHO FINO DE MESA (VINIFERA)'],
    'SUCO': df_producao[df_producao['control'] == 'SUCO'],
    'DERIVADOS': df_producao[df_producao['control'] == 'DERIVADOS']
}

print("PRODUÇÃO POR CATEGORIA (2023):")
print("=" * 40)
total_producao_2023 = 0

for categoria, dados in categorias_producao.items():
    if '2023' in dados.columns and not dados.empty:
        volume = pd.to_numeric(dados['2023'], errors='coerce').fillna(0).sum()
        total_producao_2023 += volume
        print(f"{categoria}: {volume:,.0f} litros")

print(f"\nTotal Produção 2023: {total_producao_2023:,.0f} litros")

# Análise da comercialização por categoria (2023)
categorias_comercializacao = {
    'VINHO DE MESA': df_comercializacao[df_comercializacao['control'] == 'VINHO DE MESA'],
    'VINHO FINO DE MESA': df_comercializacao[df_comercializacao['control'] == 'VINHO  FINO DE MESA'],
    'ESPUMANTES': df_comercializacao[df_comercializacao['control'] == 'ESPUMANTES '],
    'SUCO DE UVAS': df_comercializacao[df_comercializacao['control'] == 'SUCO DE UVAS']
}

print("\nCOMERCIALIZAÇÃO POR CATEGORIA (2023):")
print("=" * 45)
total_comercializacao_2023 = 0

for categoria, dados in categorias_comercializacao.items():
    if '2023' in dados.columns and not dados.empty:
        volume = pd.to_numeric(dados['2023'], errors='coerce').fillna(0).sum()
        total_comercializacao_2023 += volume
        percentual = (volume / total_comercializacao_2023 * 100) if total_comercializacao_2023 > 0 else 0
        print(f"{categoria}: {volume:,.0f} litros")

print(f"\nTotal Comercialização 2023: {total_comercializacao_2023:,.0f} litros")
print(f"Eficiência Comercial 2023: {(total_comercializacao_2023/total_producao_2023)*100:.1f}%")

PRODUÇÃO POR CATEGORIA (2023):
VINHO DE MESA: 169,762,429 litros
VINHO FINO DE MESA (VINIFERA): 46,268,556 litros
SUCO: 67,045,238 litros
DERIVADOS: 174,716,647 litros

Total Produção 2023: 457,792,870 litros

COMERCIALIZAÇÃO POR CATEGORIA (2023):
VINHO DE MESA: 187,016,848 litros
VINHO FINO DE MESA: 18,589,310 litros
ESPUMANTES: 29,381,635 litros
SUCO DE UVAS: 166,708,720 litros

Total Comercialização 2023: 401,696,513 litros
Eficiência Comercial 2023: 87.7%


## 6. Análise das Principais Variedades de Uvas

In [24]:
# Análise das variedades mais processadas (2023)
variedades_tintas = df_processamento[df_processamento['control'] == 'TINTAS'].copy()
variedades_brancas = df_processamento[df_processamento['control'] == 'BRANCASEROSADAS'].copy()

print("TOP 10 VARIEDADES TINTAS PROCESSADAS (2023):")
print("=" * 50)
if '2023' in variedades_tintas.columns:
    # Converter para numérico antes de usar nlargest
    variedades_tintas['2023_num'] = pd.to_numeric(variedades_tintas['2023'], errors='coerce').fillna(0)
    top_tintas = variedades_tintas[variedades_tintas['2023_num'] > 0].nlargest(10, '2023_num')[['cultivar', '2023_num']]
    for i, (_, row) in enumerate(top_tintas.iterrows(), 1):
        cultivar = row['cultivar']
        volume = row['2023_num']
        print(f"{i:2d}. {cultivar}: {volume:,.0f} kg")

print("\nTOP 10 VARIEDADES BRANCAS PROCESSADAS (2023):")
print("=" * 50)
if '2023' in variedades_brancas.columns:
    # Converter para numérico antes de usar nlargest
    variedades_brancas['2023_num'] = pd.to_numeric(variedades_brancas['2023'], errors='coerce').fillna(0)
    top_brancas = variedades_brancas[variedades_brancas['2023_num'] > 0].nlargest(10, '2023_num')[['cultivar', '2023_num']]
    for i, (_, row) in enumerate(top_brancas.iterrows(), 1):
        cultivar = row['cultivar']
        volume = row['2023_num']
        print(f"{i:2d}. {cultivar}: {volume:,.0f} kg")

# Evolução das principais variedades (2019-2023)
principais_variedades = ['Cabernet Sauvignon', 'Merlot', 'Chardonnay', 'Moscato Branco']

print("\nEVOLUÇÃO DAS PRINCIPAIS VARIEDADES (2019-2023):")
print("=" * 55)

for variedade in principais_variedades:
    dados_var = df_processamento[df_processamento['cultivar'].str.contains(variedade, na=False, case=False)]
    if not dados_var.empty:
        print(f"\n{variedade}:")
        for ano in ['2019', '2020', '2021', '2022', '2023']:
            if ano in dados_var.columns:
                total = pd.to_numeric(dados_var[ano], errors='coerce').fillna(0).sum()
                print(f"  {ano}: {total:,.0f} kg")

TOP 10 VARIEDADES TINTAS PROCESSADAS (2023):

TOP 10 VARIEDADES BRANCAS PROCESSADAS (2023):

EVOLUÇÃO DAS PRINCIPAIS VARIEDADES (2019-2023):

Cabernet Sauvignon:
  2019: 0 kg
  2020: 5,889,427 kg
  2021: 1,857,999 kg
  2022: 0 kg
  2023: 0 kg

Merlot:
  2019: 0 kg
  2020: 6,010,486 kg
  2021: 494,356 kg
  2022: 0 kg
  2023: 0 kg

Chardonnay:
  2019: 0 kg
  2020: 6,514,864 kg
  2021: 4,200,465 kg
  2022: 0 kg
  2023: 0 kg

Moscato Branco:
  2019: 0 kg
  2020: 11,868,851 kg
  2021: 12,100,995 kg
  2022: 0 kg
  2023: 0 kg


## 7. Análise Comparativa: Exportação vs Importação

In [25]:
# Principais países fornecedores (importação)
df_imp_paises = df_importacao.copy()
df_imp_paises['Total_2019_2023'] = 0

for ano in ['2019', '2020', '2021', '2022', '2023']:
    if ano in df_imp_paises.columns:
        df_imp_paises['Total_2019_2023'] += pd.to_numeric(df_imp_paises[ano], errors='coerce').fillna(0)

top_fornecedores = df_imp_paises[df_imp_paises['Total_2019_2023'] > 0].nlargest(10, 'Total_2019_2023')

print("TOP 10 PAÍSES FORNECEDORES - IMPORTAÇÃO (2019-2023):")
print("=" * 55)
print(f"{'Rank':<4} {'País':<25} {'Volume (L)':<15} {'%':<6}")
print("-" * 55)

total_importacao = top_fornecedores['Total_2019_2023'].sum()
for i, (_, row) in enumerate(top_fornecedores.iterrows(), 1):
    pais = row['País']
    volume = row['Total_2019_2023']
    percentual = (volume / total_importacao) * 100
    print(f"{i:<4} {pais:<25} {volume:>12,.0f} {percentual:>5.1f}%")

# Principais destinos das exportações (2019-2023)
df_exp_recente = df_exportacao[df_exportacao['País'] != 'Brasil'].copy()
df_exp_recente['Total_2019_2023'] = 0

for ano in ['2019', '2020', '2021', '2022', '2023']:
    if ano in df_exp_recente.columns:
        df_exp_recente['Total_2019_2023'] += pd.to_numeric(df_exp_recente[ano], errors='coerce').fillna(0)

top_destinos = df_exp_recente[df_exp_recente['Total_2019_2023'] > 0].nlargest(10, 'Total_2019_2023')

print("\nTOP 10 DESTINOS - EXPORTAÇÃO (2019-2023):")
print("=" * 50)
print(f"{'Rank':<4} {'País':<25} {'Volume (L)':<15} {'%':<6}")
print("-" * 50)

total_exportacao_recente = top_destinos['Total_2019_2023'].sum()
for i, (_, row) in enumerate(top_destinos.iterrows(), 1):
    pais = row['País']
    volume = row['Total_2019_2023']
    percentual = (volume / total_exportacao_recente) * 100
    print(f"{i:<4} {pais:<25} {volume:>12,.0f} {percentual:>5.1f}%")

# Balança comercial detalhada
print("\nBALANÇA COMERCIAL DETALHADA (2019-2023):")
print("=" * 50)
print(f"{'Ano':<6} {'Exportação':<12} {'Importação':<12} {'Saldo':<12} {'Status':<10}")
print("-" * 50)

for _, row in df_consolidado[df_consolidado['Ano'] >= 2019].iterrows():
    ano = int(row['Ano'])
    exp = row['Exportacao_Litros']
    imp = row['Importacao_Litros']
    saldo = exp - imp
    status = "Superávit" if saldo > 0 else "Déficit"
    
    print(f"{ano:<6} {exp:>10,.0f} {imp:>10,.0f} {saldo:>10,.0f} {status:<10}")

saldo_total = df_consolidado[df_consolidado['Ano'] >= 2019]['Balanca_Comercial'].sum()
print(f"\nSaldo Total (2019-2023): {saldo_total:,.0f} litros")

TOP 10 PAÍSES FORNECEDORES - IMPORTAÇÃO (2019-2023):
Rank País                      Volume (L)      %     
-------------------------------------------------------
1    Chile                      326,280,878  46.7%
2    Argentina                  119,286,004  17.1%
3    Portugal                   116,989,518  16.7%
4    Itália                      49,705,130   7.1%
5    Espanha                     34,839,950   5.0%
6    França                      27,035,366   3.9%
7    Uruguai                     16,945,879   2.4%
8    Africa do Sul                3,839,210   0.5%
9    Estados Unidos               2,045,346   0.3%
10   Austrália                    1,607,870   0.2%

TOP 10 DESTINOS - EXPORTAÇÃO (2019-2023):
Rank País                      Volume (L)      %     
--------------------------------------------------
1    Paraguai                    21,098,125  78.2%
2    Haiti                        2,264,528   8.4%
3    Uruguai                      1,106,164   4.1%
4    Estados Unidos       

## 8. Fatores Externos que Afetam as Exportações

In [26]:
# Análise de fatores climáticos
fatores_climaticos = {
    2009: "Ano normal - condições favoráveis",
    2010: "La Niña - chuvas excessivas no Sul",
    2011: "Seca no Nordeste - impacto moderado",
    2012: "Ano favorável - boa safra",
    2013: "Seca severa no Nordeste",
    2014: "El Niño fraco - variabilidade climática",
    2015: "El Niño forte - seca no Nordeste",
    2016: "El Niño forte - impactos na produção",
    2017: "La Niña - chuvas no Sul",
    2018: "Ano de transição - condições mistas",
    2019: "Condições normais - estabilidade",
    2020: "La Niña - boa safra no Sul",
    2021: "La Niña persistente - chuvas regulares",
    2022: "La Niña tripla - seca severa",
    2023: "El Niño - chuvas excessivas na colheita"
}

# Análise de eventos econômicos
eventos_economicos = {
    2009: "Recuperação pós-crise financeira global",
    2010: "Crescimento econômico - mercados aquecidos",
    2011: "Inflação crescente - pressões econômicas",
    2012: "Desaceleração econômica global",
    2013: "Início da desaceleração brasileira",
    2014: "Início da recessão brasileira",
    2015: "Recessão severa - queda do PIB",
    2016: "Pior ano da recessão - alta do desemprego",
    2017: "Início da recuperação econômica",
    2018: "Recuperação lenta - incertezas políticas",
    2019: "Crescimento baixo - economia estagnada",
    2020: "Pandemia COVID-19 - impacto severo",
    2021: "Recuperação parcial - inflação alta",
    2022: "Guerra Ucrânia-Rússia - pressões globais",
    2023: "Estabilização - juros altos"
}

print("FATORES EXTERNOS QUE AFETAM AS EXPORTAÇÕES:")
print("=" * 60)
print(f"{'Ano':<6} {'Exportação':<12} {'Fator Climático':<25} {'Contexto Econômico':<25}")
print("-" * 85)

for _, row in df_consolidado.iterrows():
    ano = int(row['Ano'])
    exp = row['Exportacao_Litros']
    clima = fatores_climaticos.get(ano, "N/A")[:24]
    economia = eventos_economicos.get(ano, "N/A")[:24]
    
    print(f"{ano:<6} {exp:>10,.0f} {clima:<25} {economia:<25}")

print("\nPRINCIPAIS CORRELAÇÕES IDENTIFICADAS:")
print("1. DADOS CLIMÁTICOS:")
print("   • El Niño: Geralmente reduz exportações (chuvas na colheita)")
print("   • La Niña: Pode favorecer exportações (condições mais estáveis)")
print("   • Secas severas: Impactam negativamente a produção")

print("\n2. DADOS ECONÔMICOS:")
print("   • Recessões: Reduzem demanda interna e podem favorecer exportação")
print("   • Câmbio desvalorizado: Favorece competitividade das exportações")
print("   • Crises globais: Afetam demanda internacional")

print("\n3. DADOS DEMOGRÁFICOS:")
print("   • Crescimento populacional nos mercados-alvo")
print("   • Mudanças no perfil de consumo (premium vs popular)")
print("   • Urbanização e aumento da renda per capita")

print("\n4. DADOS DE AVALIAÇÕES:")
print("   • Reconhecimento internacional ainda limitado")
print("   • Necessidade de certificações internacionais")
print("   • Competição com vinhos sul-americanos estabelecidos")

FATORES EXTERNOS QUE AFETAM AS EXPORTAÇÕES:
Ano    Exportação   Fator Climático           Contexto Econômico       
-------------------------------------------------------------------------------------
2009   25,514,198 Ano normal - condições f  Recuperação pós-crise fi 
2010    1,280,574 La Niña - chuvas excessi  Crescimento econômico -  
2011    1,214,834 Seca no Nordeste - impac  Inflação crescente - pre 
2012    5,775,376 Ano favorável - boa safr  Desaceleração econômica  
2013    9,149,059 Seca severa no Nordeste   Início da desaceleração  
2014    2,324,029 El Niño fraco - variabil  Início da recessão brasi 
2015    1,198,479 El Niño forte - seca no   Recessão severa - queda  
2016    1,786,549 El Niño forte - impactos  Pior ano da recessão - a 
2017    2,889,572 La Niña - chuvas no Sul   Início da recuperação ec 
2018    3,856,162 Ano de transição - condi  Recuperação lenta - ince 
2019    3,152,873 Condições normais - esta  Crescimento baixo - econ 
2020    4,409,717 La Niña - 

## 9. Síntese e Conclusões da Análise Descritiva

In [27]:
# Cálculos para síntese
dados_2023 = df_consolidado[df_consolidado['Ano'] == 2023].iloc[0]
crescimento_exportacao = ((dados_2023['Exportacao_Litros'] / df_consolidado.iloc[0]['Exportacao_Litros']) - 1) * 100
concentracao_top5 = (top_destinos.head(5)['Total_2019_2023'].sum() / total_exportacao_recente) * 100

print("SÍNTESE DA ANÁLISE DESCRITIVA COMPLETA")
print("=" * 50)

print("\n1. SITUAÇÃO ATUAL DO SETOR (2023):")
print(f"   • Produção total: {dados_2023['Producao_Litros']:,.0f} litros")
print(f"   • Comercialização: {dados_2023['Comercializacao_Litros']:,.0f} litros")
print(f"   • Exportação: {dados_2023['Exportacao_Litros']:,.0f} litros")
print(f"   • Taxa de exportação: {dados_2023['Taxa_Exportacao']:.2f}% da produção")
print(f"   • Balança comercial: {dados_2023['Balanca_Comercial']:,.0f} litros")
print(f"   • Países importadores ativos: {len(df_paises_ativos)}")

print("\n2. MONTANTE DE VENDAS DE EXPORTAÇÃO (2009-2023):")
print(f"   • Volume total exportado: {total_exportacao:,.0f} litros")
print(f"   • Valor estimado total: US$ {total_exportacao * preco_medio_litro:,.0f}")
print(f"   • Média anual: {total_exportacao/15:,.0f} litros")
print(f"   • Crescimento no período: {crescimento_exportacao:+.1f}%")

print("\n3. ANÁLISE POR PAÍS:")
print(f"   • Principal destino: {top_destinos.iloc[0]['País']} ({top_destinos.iloc[0]['Total_2019_2023']:,.0f} L)")
print(f"   • Concentração top 5: {concentracao_top5:.1f}% do total")
print(f"   • Mercados com potencial: Ásia, América do Norte, Europa")

print("\n4. PRINCIPAIS DESAFIOS IDENTIFICADOS:")
print("   • Baixa taxa de exportação (menos de 3% da produção)")
print("   • Déficit na balança comercial de vinhos")
print("   • Alta concentração em poucos mercados")
print("   • Dependência excessiva do mercado interno")
print("   • Falta de reconhecimento internacional")
print("   • Impactos climáticos na produção")

print("\n5. OPORTUNIDADES IDENTIFICADAS:")
print("   • Diversificação geográfica de mercados")
print("   • Investimento em variedades premium")
print("   • Certificações internacionais")
print("   • Marketing internacional coordenado")
print("   • Aproveitamento de acordos comerciais")
print("   • Desenvolvimento de produtos sustentáveis")

print("\n6. FATORES CRÍTICOS DE SUCESSO:")
print("   • Melhoria da qualidade e padronização")
print("   • Redução de custos logísticos")
print("   • Desenvolvimento de marca Brasil")
print("   • Adaptação às mudanças climáticas")
print("   • Políticas públicas de apoio")
print("   • Inovação tecnológica na produção")

# Salvar dados consolidados
df_consolidado.to_csv('analise_descritiva_setor_vitivinicola.csv', index=False)
tabela_exportacao.to_csv('tabela_exportacao_paises_completa.csv', index=False)

print("\nARQUIVOS GERADOS:")
print("• analise_descritiva_setor_vitivinicola.csv")
print("• tabela_exportacao_paises_completa.csv")
print("\nANÁLISE DESCRITIVA COMPLETA FINALIZADA")

SÍNTESE DA ANÁLISE DESCRITIVA COMPLETA

1. SITUAÇÃO ATUAL DO SETOR (2023):
   • Produção total: 915,585,740 litros
   • Comercialização: 903,876,940 litros
   • Exportação: 5,538,888 litros
   • Taxa de exportação: 0.60% da produção
   • Balança comercial: -132,173,983 litros
   • Países importadores ativos: 122

2. MONTANTE DE VENDAS DE EXPORTAÇÃO (2009-2023):
   • Volume total exportado: 83,172,462 litros
   • Valor estimado total: US$ 349,324,340
   • Média anual: 5,544,831 litros
   • Crescimento no período: -78.3%

3. ANÁLISE POR PAÍS:
   • Principal destino: Paraguai (21,098,125 L)
   • Concentração top 5: 96.5% do total
   • Mercados com potencial: Ásia, América do Norte, Europa

4. PRINCIPAIS DESAFIOS IDENTIFICADOS:
   • Baixa taxa de exportação (menos de 3% da produção)
   • Déficit na balança comercial de vinhos
   • Alta concentração em poucos mercados
   • Dependência excessiva do mercado interno
   • Falta de reconhecimento internacional
   • Impactos climáticos na produçã